In [1]:
# 데이터셋을 다운로드하고 추출
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [2]:
path.ls()

(#2) [Path('C:/Users/csjty/.fastai/data/human_numbers/train.txt'),Path('C:/Users/csjty/.fastai/data/human_numbers/valid.txt')]

In [4]:
# 합습용/검증용 파일을 열고 합친다.
lines = L()
with open(path/'train.txt') as f:lines += L(*f.readlines())
with open(path/'valid.txt') as f:lines += L(*f.readlines())    
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [6]:
# 마침표를 구분자로 한 숫자에서 다음 숫자로 넘어가는 부분을 구분
text = ' . '.join([l.strip() for l in lines])
text[:100]

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

In [7]:
# 공백을 기준으로 데이터셋을 토큰화한다.
tokens = text.split(' ')
tokens[:10]

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

In [8]:
# 수치화하려면 모든 고유 토큰(vocab)의 목록을 만들어야함
vocab = L(*tokens).unique()
vocab

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

In [9]:
# vocab의 각 요소를 조회하여 토큰에 대응하는 숫자로 변환
word2idx = {w:i for i, w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#63095) [0,1,2,1,3,1,4,1,5,1...]

In [10]:
# 이 데이터셋을 신경망에서 사용할 수 있는 형태로 바꾸자
# 이전 세 단어를 기반으로 다음 단어를 예측하도록 지정하는 방법
# 연속적인 세 단어로 이루어진 리스트를 독립변수로 그 뒤에 등장하는 단어를 종속변수로 구성한 연속적인 리스트를 만든다.
L((tokens[i:i+3], tokens[i+3]) for i in range(0, len(tokens)-4, 3))

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

In [11]:
# 모델이 실제로 사용할 수 있는 수치화된 텐서로 바꾼다.
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0, len(nums)-4, 3))
seqs

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]

In [13]:
# DataLoader 클래스를 사용하면 데이터에 쉽게 배치 단위로 접근할 수 있다. 지금은 시퀀스를 임의로 분할한다.
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [ ]:
# 이제 세단어를 입력해서 다음 단어를 예측하는 신경망 구조를 만들 수 있다. 이때 예측은 vocab에 담긴 각 단어에 대한 확률이다.
# 전형적인 선형 계층 세 개로 구성된 신경망을 사용해보자. 단 두가지를 수정할 것이다.

# 첫번째 선형계측은 첫번째 단어의 임베딩만을 두번째 선형계층은 첫번째와 두번째 단어이 임베딩을 세번째 선형계층은 첫번째, 두번째, 세번째 단어의 임베딩을 모두 활성으로 사용하도록 한다.
# 이 작업의 핵심 효과는 모든 단어가 그 앞에 오는 단어들의 문맥적인 정보에 기반해 해석된다는 점이다.

# 세 계층 각각이 같은 가중치 행렬을 사용하도록 수정한다. 한 단어가 이전 단어의 활성에 영향을 미치는 방식은 단어의 위치에 따라 바뀌어서는 안된다.
# 즉 데이터가 계측을 통해 이동함에 따라 활성값은 변경되지만 계층간의 

In [14]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden) # 임베딩 계층 (입력을 은닉 계층으로 보냄)
        self.h_h = nn.Linear(n_hidden, n_hidden) # 선형계층: 다음 단어의 활성을 생성 (은닉을 다음 은닉 계층으로 보냄)
        self.h_o = nn.Linear(n_hidden, vocab_sz) # 선형계층: 네번째 단어를 예측하는 최종 선형 계층 (은닉을 출력 계층으로 보냄)
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:, 0])))
        h = h + self.i_h(x[:, 1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:, 2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [16]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.856449,1.986673,0.464464,00:02
1,1.381384,1.756919,0.473734,00:02
2,1.415422,1.641688,0.493701,00:02
3,1.371177,1.654577,0.479201,00:02


In [ ]:
n, counts = 0, torch.zeros(len(vocab))
for x, y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
